In [56]:
%pip install redis
%pip install redisearch
%pip install llama-cpp-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [57]:
from langchain.embeddings import LlamaCppEmbeddings

#from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

llama = LlamaCppEmbeddings(model_path="Users/josi/Llama2_weights/llama-7b.ggmlv3.q4_K_M.bin")

text = "This is text"

query_result = llama.embed_query(text)
doc_result = llama.embed_documents([text])

ValidationError: 1 validation error for LlamaCppEmbeddings
__root__
  Could not load Llama model from path: Users/josi/Llama2_weights/llama-7b.ggmlv3.q4_K_M.bin. Received error Model path does not exist: Users/josi/Llama2_weights/llama-7b.ggmlv3.q4_K_M.bin (type=value_error)

In [58]:
import redis
import logging
from langchain.vectorstores.redis import Redis
import uuid

In [94]:
#embeddings = OpenAIEmbeddings()

#connection = redis.Redis()
#connection.ping()

r = redis.Redis(host='localhost', port=6379, decode_responses=True, db=0)
#further setup with tsl?
# Test the connection
ping_response = r.ping()
if ping_response:
    print("Connected to Redis server successfully!")
else:
    print("Failed to connect to Redis server.")

#class Redis(object):#
#    def __init__(self, host='localhost', port=6379,db=0, password=None, socket_timeout=None)

Connected to Redis server successfully!


In [96]:
#create connection to redis cloud server

r = redis.Redis(
  host='redis-17335.c304.europe-west1-2.gce.cloud.redislabs.com',
  port=17335,
  password='oUkqSduoXftkjL0DOZ0rZ1mqOH5urQZn', decode_responses=True)
ping_response = r.ping()
if ping_response:
    print("Connected to Redis Cloud server successfully!")
else:
    print("Failed to connect to Redis Cloud server.")

Connected to Redis Cloud server successfully!


In [60]:
#run redis-server in terminal
#redis-cli for host:port pair -> test connectivity with "PING"
#to close: redis-cli shutdown

#MSET to set multiple key:value pairs and MGET to get multple values for keys

In [97]:
file_path = './index.txt'

with open(file_path, 'rt') as file:
    lines = file.readlines()

#for line in lines:
#    print(line)

#clear previous data from redis db
#r.flushdb()

# Store each line from the Llama text file as a separate key-value pair in Redis
#for index, line in enumerate(lines):
#    r.set(f'line:{index}', line.strip())

# Store each line from the Llama text file as a separate key-value pair in Redis
for index, line in enumerate(lines):
    key = f'line:{index}'
    value = line.strip()
    r.set(key, value)

###store key:value pairs in new txt file
# Read the contents of the Redis
values = r.scan_iter()

# Write the values to a new txt file
with open('new_file.txt', 'w') as file:
    for value in values:
        data_type = r.type(value)
        
        if data_type == 'set':
            stored_lines = r.smembers(value)
            for stored_line in stored_lines:
                file.write(f'{value}: {stored_line}\n')
        elif data_type == 'string':
            stored_line = r.get(value)
            if isinstance(stored_line, bytes):
                stored_line = stored_line.decode('utf-8')
                file.write(f'{value}: {stored_line}\n')
            file.write(f'{value}: {stored_line}\n')
        else:
            print(f"Warning: Ignoring key '{value}' of unsupported data type '{data_type}'")

# Delete the keys with unsupported data types
for value in values:
    data_type = r.type(value)
    if data_type not in ['set', 'string']:
        r.delete(value)
        print(f"Deleted key '{value}' of unsupported data type '{data_type}'")

In [98]:
# Print what you have in the index.txt file
for index, line in enumerate(lines):
    key = f'line:{index}'
    stored_line = r.get(key)
    print(stored_line)

# Create a unique key for each text
key = str(uuid.uuid4())

value = stored_line

# Set the value of the key to the text
r.set(key, value)

stored_text = r.get(key)
print(stored_text)



r.scan()
# Read the contents of the Llama text file
with open('./index.txt', 'r') as file:
    lines = file.readlines()

print(lines)

Lorem ipsum dolor sit amet, consectetur adipiscing elit.
Praesent dapibus, elit quis commodo pharetra, magna ante convallis nulla, id elementum eros nulla quis eros.
Cras pretium hendrerit dui, sagittis congue nisl tempus sit amet. Vestibulum fermentum, velit ut aliquam blandit, diam nunc tincidunt sapien, id luctus erat ipsum sit amet mauris. Nunc viverra fermentum eros, a feugiat arcu laoreet et. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. In tincidunt turpis vitae felis volutpat, a eleifend nibh facilisis. Phasellus et sapien metus. Vestibulum sem turpis, hendrerit quis purus volutpat, scelerisque venenatis ante. Sed tristique dolor sit amet dignissim luctus. Vivamus ultricies est non volutpat sagittis. Cras commodo euismod mi. Aliquam mollis iaculis blandit.
Cras pretium hendrerit dui, sagittis congue nisl tempus sit amet. Vestibulum fermentum, velit ut aliquam blandit, diam nunc tincidunt sapien, id luctus erat ipsum sit amet mauri

In [63]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Document loader
loader = TextLoader("./index.txt")
documents = loader.load()

with open("./index.txt") as f:
    document = f.read()

# normal text splitter
raw_documents = TextLoader("./index.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(raw_documents)

# Recursive text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,
)

print(documents)

[Document(page_content='Lorem ipsum dolor sit amet, consectetur adipiscing elit. \nPraesent dapibus, elit quis commodo pharetra, magna ante convallis nulla, id elementum eros nulla quis eros. \nCras pretium hendrerit dui, sagittis congue nisl tempus sit amet. Vestibulum fermentum, velit ut aliquam blandit, diam nunc tincidunt sapien, id luctus erat ipsum sit amet mauris. Nunc viverra fermentum eros, a feugiat arcu laoreet et. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. In tincidunt turpis vitae felis volutpat, a eleifend nibh facilisis. Phasellus et sapien metus. Vestibulum sem turpis, hendrerit quis purus volutpat, scelerisque venenatis ante. Sed tristique dolor sit amet dignissim luctus. Vivamus ultricies est non volutpat sagittis. Cras commodo euismod mi. Aliquam mollis iaculis blandit.', metadata={'source': './index.txt'})]


In [ ]:
#store and retrieve dict
r.hset('user-session:123', mapping={
    'name': 'John',
    "surname": 'Smith',
    "company": 'Redis',
    "age": 29
})
r.hgetall('user-session:123')

r.mset({"Croatia": "Zagreb", "Bahamas": "Nassau"})
r.get("Bahamas")

{'name': 'John', 'surname': 'Smith', 'company': 'Redis', 'age': '29'}

In [64]:
metadata = [
    {
        "user": "john",
        "age": 18,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "derrick",
        "age": 45,
        "job": "doctor",
        "credit_score": "low",
    },
    {
        "user": "nancy",
        "age": 94,
        "job": "doctor",
        "credit_score": "high",
    },
    {
        "user": "tyler",
        "age": 100,
        "job": "engineer",
        "credit_score": "high",
    },
    {
        "user": "joe",
        "age": 35,
        "job": "dentist",
        "credit_score": "medium",
    },
]

In [65]:
logging.basicConfig()

In [ ]:
r.execute_command("MODULE", "LOAD", "/path/to/your/redisearch.so")

In [ ]:
r.execute_command("MODULE", "LIST")

In [74]:
###json elements 
#from redisearch import Path
from redisearch import Client, TextField, IndexDefinition, TagField,NumericField
#from redisearch import IndexDefinition, IndexType
#from redisearch import NumericFilter, Query


from redisearch import Client, TextField, IndexDefinition
import datetime
import os
from redis import ResponseError

client = Client("index-document")
#definition = IndexDefinition(prefix=’doc:’) -> available for redisearch 2.0

START_TIME = datetime.datetime.now().strftime("%Y-%m-%d-%H:%M.%S")

client.redis.set("start-time", START_TIME)

try:
    client.info()
except ResponseError:
    print("Index does not exist. We need to create it!")

#set-up
user1 = {
    "name": "Paul John",
    "email": "paul.john@example.com",
     "city":"Berlin",
    "age": 42
}
user2 = {
    "name": "Eden Zamir",
    "email": "eden.zamir@example.com",
    "city":"Berlin",
    "age": 29
}

schema = (
    TextField("$.name", as_name="name"), 
    TagField("$.city", as_name="city"), 
    NumericField("$.age", as_name="age")
)

rs = r.ft("idx:users")
rs.create_index(
    schema,
    definition=IndexDefinition(
        prefix=["user:"], index_type=IndexType.JSON
    )
)

r.json().set("user:1", Path.root_path(), user1)
r.json().set("user:2", Path.root_path(), user2)

#Search
res = rs.search(
    Query("Paul @age:[30 40]")
)

rs.search(
    Query("Paul").return_field("$.city", as_field="city")
).docs

Index does not exist. We need to create it!


AttributeError: 'Redis' object has no attribute 'ft'

In [ ]:
schema = (
    TextField("$.brand", as_name="brand"),
    TextField("$.model", as_name="model"),
    TextField("$.description", as_name="description"),
    NumericField("$.price", as_name="price"),
    TagField("$.condition", as_name="condition"),
)

index = r.ft("idx:bicycle")
index.create_index(
    schema,
    definition=IndexDefinition(prefix=["bicycle:"], index_type=IndexType.JSON),
)

for bid, bicycle in enumerate(bicycles):
    r.json().set(f"bicycle:{bid}", Path.root_path(), bicycle)

res = index.search(Query("*"))
print("Documents found:", res.total)

res = index.search(Query("@model:Jigger"))
print(res)

In [69]:
#https://faun.pub/redisearch-using-python-client-3581309e3475
###Creating the index definition and schema
try:
    client \
        .create_index(
            (TextField("title"), TextField("body")),
            definition=definition
        )
except:
    pass


###add document
# path of the folder where documents are stored
folder = "/Users/josi/awt-pjws23-24-LLM-2"
# indexing each documents in the folder
for file in os.listdir(folder):
    filepath = os.path.join(folder, file)
    f = open(filepath, 'r')
    client.redis.hset('doc:'+file,
                mapping={
                'title': file,
                'body': f.read()
            })
    f.close()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 556: invalid start byte